<a href="https://colab.research.google.com/github/clem-mont-inha/DBWeek9/blob/main/Week9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install ipython-sql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.2 MB/s eta 0:00:00


In [11]:
import sqlite3
import pandas as pd

%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [12]:
conn = sqlite3.connect('lab_session.db')
c = conn.cursor()

In [13]:
commands = [
"""CREATE TABLE IF NOT EXISTS students(student_id INTEGER PRIMARY KEY, first_name TEXT, last_name TEXT, major TEXT);""" ,
"""INSERT INTO students(first_name, last_name, major) VALUES('John', 'Doe', 'Computer Science'), ('Jane', 'Doe', 'Mathematics'), ('Mike', 'Jordan', 'Physics');""" ,
"""CREATE TABLE IF NOT EXISTS courses(course_id INTEGER PRIMARY KEY, course_name TEXT, department_id INTEGER, instructor_id INTEGER, FOREIGN KEY(department_id) REFERENCES departments (department_id), FOREIGN KEY (instructor_id) REFERENCES instructors (instructor_id));""" ,
"""INSERT INTO courses(course_name, department_id, instructor_id) VALUES ('Database Systems', 1, 1), ('Data Structures', 1, 2), ('Linear Algebra', 2, 3), ('Quantum Mechanics', 3 , 4);""",
"""CREATE TABLE IF NOT EXISTS enrollments(enrollment_id INTEGER PRIMARY KEY, student_id INTEGER, course_id INTEGER, FOREIGN KEY (student_id) REFERENCES students (student_id), FOREIGN KEY (course_id) REFERENCES courses (course_id));""",
"""INSERT INTO enrollments(student_id, course_id) VALUES (1, 1), (1, 2), (2, 3), (3, 4);""",
 """
 CREATE TABLE IF NOT EXISTS instructors (
 instructor_id INTEGER PRIMARY KEY ,
 first_name TEXT ,
 last_name TEXT ,
 department_id INTEGER ,
 FOREIGN KEY ( department_id ) REFERENCES departments (
department_id )
 );
 """ ,
 """
 INSERT INTO instructors ( first_name , last_name , department_id )
 VALUES (' Robert ', 'Smith ', 1) ,
 (' Susan ', 'Johnson ', 2) ,
 (' Michael ', 'Brown ', 3) ,
 (' Elizabeth ', 'Taylor ', 4);
 """ ,
 """
 CREATE TABLE IF NOT EXISTS departments (
 department_id INTEGER PRIMARY KEY ,
 department_name TEXT
 );
 """ ,
 """
 INSERT INTO departments ( department_name )
 VALUES (' Computer Science '),
 (' Mathematics '),
 (' Physics '),
 (' Chemistry ');
 """
]
for command in commands:
  c.execute(command)
conn.commit()
conn.close()

In [15]:
conn = sqlite3.connect ('lab_session.db')
inner_join_query = """
SELECT students.first_name AS student_name, courses.course_name FROM students
INNER JOIN enrollments ON students.student_id = enrollments.student_id
INNER JOIN courses ON enrollments.course_id = courses.course_id ;
"""

inner_join_df = pd.read_sql_query(inner_join_query, conn)
print(inner_join_df)

  student_name        course_name
0         John   Database Systems
1         John    Data Structures
2         Jane     Linear Algebra
3         Mike  Quantum Mechanics


In [16]:
left_join_query = """
SELECT students.first_name AS student_name, courses.course_name
FROM students
LEFT JOIN enrollments ON students.student_id = enrollments.student_id
LEFT JOIN courses ON enrollments.course_id = courses.course_id ;
"""
left_join_df = pd.read_sql_query(left_join_query, conn)
print(left_join_df)

  student_name        course_name
0         John   Database Systems
1         John    Data Structures
2         Jane     Linear Algebra
3         Mike  Quantum Mechanics


In [17]:
full_join_query_1 = """
WITH RECURSIVE full_join AS (
SELECT students.first_name AS student_name , courses.course_name
FROM students
LEFT JOIN enrollments ON students.student_id = enrollments.student_id
LEFT JOIN courses ON enrollments.course_id = courses.course_id
UNION ALL
SELECT students.first_name AS student_name , courses.course_name
FROM courses
LEFT JOIN enrollments ON courses.course_id = enrollments.course_id
LEFT JOIN students ON enrollments.student_id = students.student_id
WHERE students.student_id IS NULL
)
SELECT * FROM full_join ;
"""

full_join_df_1 = pd.read_sql_query ( full_join_query_1 , conn )
print(full_join_df_1)

  student_name        course_name
0         John   Database Systems
1         John    Data Structures
2         Jane     Linear Algebra
3         Mike  Quantum Mechanics
